In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


In [2]:
# ========== UPDATE THESE PATHS ==========

# Path to your dataset (CSV or Excel)
DATA_FILE = r'F:\7_working\1_Hackathon_UIDAI\Cleaned State Wise Data - Total Enrollments in 2025.csv'  # or 'your_monthly_data.xlsx'

# Path to your mapping.json file
MAPPING_FILE = 'mapping.json'

# Path to your india_states.geojson file
GEOJSON_FILE = 'india_states.geojson'

# Column names in your dataset
STATE_COLUMN = 'State'           # Column with state names/codes
MONTH_COLUMN = 'Month'           # Column with month information
METRIC_COLUMN = "Demo_age_5_17"   # Column with your metric (or use list for multiple metrics)

# If you have multiple metrics, list them here:
# METRIC_COLUMNS = ['metric1', 'metric2']

# Output directory for saving individual month images
OUTPUT_DIR = 'monthly_heatmaps'

# ========================================

In [3]:
# Load your data
if DATA_FILE.endswith('.csv'):
    df = pd.read_csv(DATA_FILE)
elif DATA_FILE.endswith('.xlsx'):
    df = pd.read_excel(DATA_FILE)
else:
    raise ValueError("Data file must be CSV or Excel format")

print(f"✓ Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head(10)

✓ Data loaded: 306 rows, 4 columns

Columns: ['State', 'Month', 'Demo_age_5_17', 'Demo_age_17+']

First few rows:


,State,Month,Demo_age_5_17,Demo_age_17+
0,Andaman and Nicobar Islands,2025-03,114.0,895.0
1,Andaman and Nicobar Islands,2025-09,93.0,1010.0
2,Andaman and Nicobar Islands,2025-10,62.0,611.0
3,Andaman and Nicobar Islands,2025-11,114.0,1708.0
4,Andaman and Nicobar Islands,2025-12,156.0,2059.0
5,Andaman and Nicobar Islands,2026-01,13.0,101.0
6,Andhra Pradesh,2026-01,3714.0,25522.0
7,Andhra Pradesh,2025-12,54331.0,421990.0
8,Andhra Pradesh,2025-11,61949.0,320340.0
9,Andhra Pradesh,2025-10,37112.0,175479.0


In [4]:
# Convert month column to datetime
# Adjust the format string based on your date format
# Common formats:
#   'March 2025' -> '%B %Y'
#   '2025-03' -> '%Y-%m'
#   '03/2025' -> '%m/%Y'

df['month_date'] = pd.to_datetime(df[MONTH_COLUMN], format='%Y-%m')  # Adjust format as needed

# Sort by month
df = df.sort_values('month_date')

# Get unique months in chronological order
unique_months = df['month_date'].dt.strftime('%B %Y').unique()

print(f"✓ Found {len(unique_months)} unique months:")
for i, month in enumerate(unique_months, 1):
    count = len(df[df['month_date'].dt.strftime('%B %Y') == month])
    print(f"  {i}. {month} - {count} records")

✓ Found 10 unique months:
  1. March 2025 - 35 records
  2. April 2025 - 24 records
  3. May 2025 - 24 records
  4. June 2025 - 20 records
  5. July 2025 - 23 records
  6. September 2025 - 36 records
  7. October 2025 - 36 records
  8. November 2025 - 36 records
  9. December 2025 - 36 records
  10. January 2026 - 36 records


In [5]:
# Load state name mapping
with open(MAPPING_FILE, 'r') as f:
    state_mapping = json.load(f)

print("✓ State mapping loaded")
print(f"  Total mappings: {len(state_mapping)}")

# Load GeoJSON
with open(GEOJSON_FILE, 'r') as f:
    geojson_data = json.load(f)

# Convert to GeoDataFrame
india_gdf = gpd.GeoDataFrame.from_features(geojson_data['features'])
india_gdf['state_name'] = india_gdf['ST_NM']

print("✓ GeoJSON loaded")
print(f"  Total states/territories: {len(india_gdf)}")

✓ State mapping loaded
  Total mappings: 36
✓ GeoJSON loaded
  Total states/territories: 36


In [6]:
# Apply mapping to standardize state names
df['state_standardized'] = df[STATE_COLUMN].map(state_mapping)

# Check for unmapped states
unmapped = df[df['state_standardized'].isna()][STATE_COLUMN].unique()
if len(unmapped) > 0:
    print("⚠ Warning: These states were not found in mapping:")
    for state in unmapped:
        print(f"  - {state}")
else:
    print("✓ All states mapped successfully!")

# Show sample of mapped data
print("\nSample mapped data:")
df[[STATE_COLUMN, 'state_standardized', MONTH_COLUMN, METRIC_COLUMN]].head(10)

✓ All states mapped successfully!

Sample mapped data:


,State,state_standardized,Month,Demo_age_5_17
0,Andaman and Nicobar Islands,Andaman & Nicobar,2025-03,114.0
59,Chhattisgarh,Chhattisgarh,2025-03,12411.0
62,Dadra and Nagar Haveli and Daman and Diu,Dadra and Nagar Haveli and Daman and Diu,2025-03,144.0
77,Delhi,Delhi,2025-03,16216.0
78,Goa,Goa,2025-03,1089.0
92,Gujarat,Gujarat,2025-03,52799.0
102,Haryana,Haryana,2025-03,15202.0
112,Himachal Pradesh,Himachal Pradesh,2025-03,3371.0
122,Jammu and Kashmir,Jammu & Kashmir,2025-03,8495.0
130,Jharkhand,Jharkhand,2025-03,16306.0


In [7]:
# Create output directory if it doesn't exist
output_path = Path(OUTPUT_DIR)
output_path.mkdir(exist_ok=True)

print(f"✓ Output directory ready: {OUTPUT_DIR}/")

✓ Output directory ready: monthly_heatmaps/


In [8]:
def create_monthly_heatmap(month, metric_column, save=True, show=True, 
                          cmap='YlOrRd', figsize=(12, 14), vmin=None, vmax=None):
    """
    Create a heatmap for a specific month
    
    Parameters:
    -----------
    month : str
        Month to visualize (e.g., 'March 2025')
    metric_column : str
        Column name of the metric to visualize
    save : bool
        Whether to save the image
    show : bool
        Whether to display the image
    cmap : str
        Colormap name
    figsize : tuple
        Figure size (width, height)
    vmin, vmax : float
        Min and max values for color scale (for consistency across months)
    """
    
    # Filter data for this month
    month_data = df[df['month_date'].dt.strftime('%B %Y') == month].copy()
    
    # Merge with geodataframe
    india_merged = india_gdf.merge(
        month_data,
        left_on='state_name',
        right_on='state_standardized',
        how='left'
    )
    
    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    
    # Plot the heatmap
    india_merged.plot(
        column=metric_column,
        ax=ax,
        legend=True,
        cmap=cmap,
        edgecolor='black',
        linewidth=0.5,
        vmin=vmin,
        vmax=vmax,
        missing_kwds={
            "color": "lightgrey",
            "edgecolor": "black",
            "hatch": "///",
            "label": "No data"
        },
        legend_kwds={
            'label': metric_column.replace('_', ' ').title(),
            'orientation': "vertical",
            'shrink': 0.7
        }
    )
    
    # Title with month and data count
    data_count = india_merged[metric_column].notna().sum()
    title = f'India {metric_column.replace("_", " ").title()} - {month}\n({data_count} states/territories with data)'
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    
    # Save if requested
    if save:
        # Create safe filename
        safe_month = month.replace(' ', '_')
        safe_metric = metric_column.replace(' ', '_')
        filename = f"{OUTPUT_DIR}/{safe_metric}_{safe_month}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight', facecolor='white')
    
    # Show if requested
    if show:
        plt.show()
    else:
        plt.close()
    
    return fig, ax

print("✓ Heatmap function defined!")

✓ Heatmap function defined!


In [9]:
# Calculate global min/max for consistent color scale across all months
vmin = df[METRIC_COLUMN].min()
vmax = df[METRIC_COLUMN].max()

print(f"Generating heatmaps for all {len(unique_months)} months...")
print(f"Color scale: {vmin:.2f} to {vmax:.2f}\n")

for i, month in enumerate(unique_months, 1):
    print(f"[{i}/{len(unique_months)}] Creating heatmap for {month}...")
    create_monthly_heatmap(
        month=month,
        metric_column=METRIC_COLUMN,
        save=True,
        show=False,  # Don't show while generating all (too many plots)
        cmap='YlOrRd',
        vmin=vmin,
        vmax=vmax
    )

print(f"\n✓ All {len(unique_months)} heatmaps saved to '{OUTPUT_DIR}/' directory!")

Generating heatmaps for all 10 months...
Color scale: 6.00 to 265321.00

[1/10] Creating heatmap for March 2025...
[2/10] Creating heatmap for April 2025...
[3/10] Creating heatmap for May 2025...
[4/10] Creating heatmap for June 2025...
[5/10] Creating heatmap for July 2025...
[6/10] Creating heatmap for September 2025...
[7/10] Creating heatmap for October 2025...
[8/10] Creating heatmap for November 2025...
[9/10] Creating heatmap for December 2025...
[10/10] Creating heatmap for January 2026...

✓ All 10 heatmaps saved to 'monthly_heatmaps/' directory!


In [ ]:
# Create interactive widget
def update_heatmap(month_index):
    """
    Update function for the slider widget
    """
    clear_output(wait=True)
    month = unique_months[month_index]
    create_monthly_heatmap(
        month=month,
        metric_column=METRIC_COLUMN,
        save=False,
        show=True,
        cmap='YlOrRd',
        vmin=vmin,
        vmax=vmax,
        figsize=(14, 16)
    )

# Create slider
month_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(unique_months) - 1,
    step=1,
    description='Month:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Create month label widget
month_label = widgets.Label(value=unique_months[0])

def update_label(change):
    month_label.value = unique_months[change['new']]

month_slider.observe(update_label, 'value')

# Display widgets
print("\n" + "="*80)
print("INTERACTIVE HEATMAP SLIDER")
print("="*80)
print("Use the slider below to scroll through months!\n")

display(widgets.VBox([month_label, month_slider]))

# Create interactive output
interactive_plot = widgets.interactive_output(update_heatmap, {'month_index': month_slider})
display(interactive_plot)


INTERACTIVE HEATMAP SLIDER
Use the slider below to scroll through months!



Output()

In [11]:
# Uncomment to install imageio if needed
# !pip install imageio

import imageio
from PIL import Image

def create_animation(duration_per_frame=1000):
    """
    Create an animated GIF from all monthly heatmaps
    
    Parameters:
    -----------
    duration_per_frame : int
        Milliseconds to display each frame (1000 = 1 second)
    """
    print("Creating animated GIF...")
    
    images = []
    for month in unique_months:
        safe_month = month.replace(' ', '_')
        safe_metric = METRIC_COLUMN.replace(' ', '_')
        filename = f"{OUTPUT_DIR}/{safe_metric}_{safe_month}.png"
        images.append(imageio.imread(filename))
    
    # Save as GIF
    output_gif = f"{OUTPUT_DIR}/animation_{METRIC_COLUMN}.gif"
    imageio.mimsave(output_gif, images, duration=duration_per_frame)
    
    print(f"✓ Animation saved: {output_gif}")
    return output_gif

# Uncomment to create animation
# create_animation(duration_per_frame=1000)  # 1 second per month

In [12]:
print("="*80)
print("SUMMARY")
print("="*80)

print(f"\nDataset Information:")
print(f"  Total records: {len(df)}")
print(f"  Unique states: {df['state_standardized'].nunique()}")
print(f"  Unique months: {len(unique_months)}")
print(f"  Date range: {unique_months[0]} to {unique_months[-1]}")

print(f"\nMetric Statistics ({METRIC_COLUMN}):")
print(df[METRIC_COLUMN].describe())

print(f"\nFiles Generated:")
print(f"  Output directory: {OUTPUT_DIR}/")
print(f"  Total heatmap images: {len(unique_months)}")

# Count states with data per month
print(f"\nStates with data per month:")
for month in unique_months:
    month_data = df[df['month_date'].dt.strftime('%B %Y') == month]
    count = month_data['state_standardized'].nunique()
    print(f"  {month}: {count} states")

print("\n" + "="*80)
print("✓ ALL TASKS COMPLETED SUCCESSFULLY!")
print("="*80)

SUMMARY

Dataset Information:
  Total records: 306
  Unique states: 36
  Unique months: 10
  Date range: March 2025 to January 2026

Metric Statistics (Demo_age_5_17):
count       306.000000
mean      13865.349673
std       24114.629342
min           6.000000
25%         674.000000
50%        4576.500000
75%       16419.250000
max      265321.000000
Name: Demo_age_5_17, dtype: float64

Files Generated:
  Output directory: monthly_heatmaps/
  Total heatmap images: 10

States with data per month:
  March 2025: 35 states
  April 2025: 24 states
  May 2025: 24 states
  June 2025: 20 states
  July 2025: 23 states
  September 2025: 36 states
  October 2025: 36 states
  November 2025: 36 states
  December 2025: 36 states
  January 2026: 36 states

✓ ALL TASKS COMPLETED SUCCESSFULLY!
